## Preparations
The data which we will use to create holdings and items are contianed within the 952 (item) and 852/866 (holding) fields of the bibliographic MARC export. Some bibs will contain mutliple occurences of 952/852/866.
<br><br>
We can use (for example) the service task MARCFieldsToTSV to create delimited files from the MARC file.
<br><br>
Run python ~/code/service_tools/main_cli.py MARCFieldsToTSV to learn more about the script arguments. The script will create one row per occurence of the leftmost MARC tag in the MARC tags argument("952" in as in "952,999,852"). In the result file, the _0_ in 952_0_a, is used to create unique column headers when there are multiple occurences of the same field in a record.

### Create a tsv file
1. Note that you might run into problems if the MARC file contains tabs (!). If that is the case, you can do a find/replace in the MARC file before you run the script to create the delimited file.
2. Run the bib MARC file through MARCFieldsToTSV to create a tsv file containing one row per 952 field in the bibs file. Also include the 999 field which contains the bib number, as well as the 852 and 866. The command will look something like this:

    ```
    python ~/code/service_tools/main_cli.py MARCFieldsToTSV ~/client_data/my_library/iterations/bibs_test/source_data/instances/my_library_bibs.mrc ~/client_data/migration_whu/iterations/bibs_test/source_data/items/my_library_items_from_952.tsv "952,999,852,866" no
    ```
3. To make sure the expected items were generated, compare the number of rows in your new file with the number of 952s in your bib MARC file.


## Open and inspect file
Start by reading the file into a Pandas dataframe. If there is an encoding issue, use encoding="unicode_escape" parametre to read the file anyway.

In [ ]:
import csv
import pandas as pd
import numpy as np
import time


In [ ]:
folder = "~/client_data/my_library/iterations/bibs_test/source_data/items"
# Replace this with your latest file
filename = f"/items_from_952.tsv"

delimiter = "\t"


data = pd.read_csv(f"{folder}{filename}", dtype=object, delimiter=delimiter, na_filter=False)



In [ ]:
# Sort the colmns (for visiiblity it's nice to have bub ID 999 to the left)
data = data.reindex(sorted(data.columns, reverse=True),  axis=1)
data.info(verbose=True)
data.head(3)


### Rename columns
Use this Koha 952 map: https://wiki.koha-community.org/wiki/Holdings_data_fields_(9xx)#MARC21_Holding_field_.28952.29

In [ ]:
map_marc_to_names = {
    "952_0_9": "itemnumber",
    "952_0_p": "barcode",
    "952_0_d": "dateaccessioned",
    "952_0_e": "booksellerid",
    "952_0_a": "homebranch",
    "952_0_g": "price",
    "952_0_v": "replacementprice",
    "952_0_w": "replacementpricedate",
    "952_0_s": "datelastborrowed",
    "952_0_r": "datelastseen",
    "952_0_j": "stack",
    "952_0_7": "notforloan",
    "952_0_4": "damaged",
    "952_0_1": "itemlost",
    "952_0_0": "wthdrawn",
    "952_0_o": "itemcallnumber",
    "952_0_l": "issues",
    "952_0_m": "renewals",
    "952_0_n": "reserves",
    "952_0_5": "restricted",
    "952_0_z": "itemnotes",
    "952_0_b": "holdingbranch",
    "952_0_c": "location",
    "952_0_q": "onloan",
    "952_0_2": "cn_source",
    "952_0_6": "cn_sort",
    "952_0_8": "ccode",
    "952_0_3": "materials",
    "952_0_u": "uri",
    "952_0_y": "itype",
    "952_0_h": "enumchron",
    "952_0_t": "copynumber",
    "952_0_i": "stocknumber",
}


In [ ]:
data.rename(map_marc_to_names, axis=1, inplace=True)
print(data.columns)


In [ ]:
#Print column value counts
print(data.columns)
for column in data.columns:
    vcdf = pd.DataFrame(data[column].value_counts())
    print(f"\n{vcdf.head(20).to_markdown()}")

## Save to a new file in the same folder

In [ ]:
timestamp = time.strftime("%Y%m%d-%H%M%S")

data.to_csv(
    f"{folder}{filename[:-4]}_prepped_{timestamp}.tsv",
    sep="\t",
    index=False,
    quoting=csv.QUOTE_NONE,
    escapechar="\\",
)


## Next steps...